# Assignment  week 2

## Notebook made by   (If not filled in correctly: 0 pts for assignment)

__Name(s)__: 

__Student id(s)__ : 

### Pledge (taken from [Coursera's Honor Code](https://www.coursera.org/about/terms/honorcode) )



Put here a selfie with your photo where you hold a signed paper with the following text: (if this is team work, put two selfies here). The link must be to some place on the web, not to a local file. **Assignments without the selfies will not be graded and receive 0 points.**

> My answers to homework, quizzes and exams will be my own work (except for assignments that explicitly permit collaboration).

>I will not make solutions to homework, quizzes or exams available to anyone else. This includes both solutions written by me, as well as any official solutions provided by the course staff.

>I will not engage in any other activities that will dishonestly improve my results or dishonestly improve/hurt the results of others.

<img width=30% src='http://oi63.tinypic.com/2q3srh3.jpg'/>
<img src="http://i63.tinypic.com/2itj1ic.jpg" border="0">

## Questions

#### Book

* Chap 6: 6.8 to 6.13, 6.14 to 6.17. 
 
#### Programming

* [MakeInvertedIndex](MakeInvertedIndex1516.html): Exercises 8.1 to 8.6 about Boolean Search Engines and Exercises 9.1 to 9.5 about Ranked Retrieval 
 


# Programming

For the programming part, we'll first copy and paste what we need from Assignment 1.

In [1]:
from bs4 import BeautifulSoup
from nltk.tokenize import WhitespaceTokenizer
from collections import Counter, defaultdict
import os
import pickle
import re

CORPUS = "Shakespeare"

def normalize(word, keep_capitals):
    """
    Normalize capital letters and punctuation
    """
    if (keep_capitals and not word == word.upper()) or not keep_capitals:
        word = word.lower()
    word = re.sub(r'[^\w\s]', '', word)
    return word

def prepare_data(folder, new_folder, keep_capitals, byte):
    """
    Prepare a text corpus for the process of
    making an inverted index.
    """
    # Create a directory for the processed files
    if not os.path.exists(new_folder):
        os.makedirs(new_folder)

    # Prepare each file one by one
    for filename in os.listdir(folder):
        if not os.path.exists(os.path.join(new_folder, filename[:-4] + ".pkl")):
            # Strip xml tags
            if byte:
                xmlfile = BeautifulSoup(open(os.path.join(folder, filename), 'rb'),"xml")
            else:
                xmlfile = BeautifulSoup(open(os.path.join(folder, filename), 'r'),"xml")
            textfile = xmlfile.get_text()

            # Split the text on whitespaces
            beautifile = WhitespaceTokenizer().tokenize(textfile)

            # Normalize text for capital letters, non-alphabetical tokens and whitespace
            finalfile = [ normalize(word, keep_capitals)
                          for word 
                          in beautifile 
                         ]
            new_filename = os.path.join(new_folder, filename[:-4] + ".pkl")
            savefile = open(new_filename, 'wb')
            pickle.dump(finalfile, savefile)


In [3]:
def index_collection(folder):
    """
    Create an inverted index given a folder 
    containig a text corpus.
    """
    index= defaultdict(Counter) # initialize MyIndex
    for filename in os.listdir(folder):
        text = pickle.load(open(os.path.join(folder,filename),'rb'))
        
        # Update the index with each token
        for w in text:    
            index[w][filename[:-4]]+=1   
    if '' in index.keys():
        del index['']
    return index

In [34]:
prepare_data(CORPUS, CORPUS + "New", 0, 0)
index_org = index_collection(CORPUS + "New")

In [6]:
def calculate_doc_freq(index, word):
    """
    Calculate in how many documents a given 
    term occurs, based on the inverted index.
    """
    return len(index[word])
    
def calculate_corp_freq(index, word):
    """
    Calculate how often a term occurs in a 
    corpus, based on the inverted index.
    """
    corpus_frequency = 0
    
    for document in index[word]:
        corpus_frequency += index[word][document]
        
    return corpus_frequency

def add_frequencies(index):
    for word in index:
        original_counter = index[word]
        document_frequency = calculate_doc_freq(index, word)
        corpus_frequency = calculate_corp_freq(index, word)
        index[word] = {"counter" : original_counter, 
                       "doc_freq" : document_frequency, 
                       "corp_freq" : corpus_frequency
                      }
    return index

In [35]:
index_with_freq = add_frequencies(index_org)

#### making a list of all documents

Will be useful for negation.

In [57]:

def make_doc_set(index_with_frequencies):
    
    doc_list = []
    for token in index_with_freq:
        doc_list.extend(list(index_with_freq[token]['counter'].elements()))
    return set(doc_list)

In [59]:
doc_set = make_doc_set(index_with_freq)
print(doc_set)

{'tempest', 'lear', 'two_gent', 'hamlet', 'rich_iii', 'r_and_j', 'hen_v', 'troilus', 'cymbelin', 'all_well', 'lll', 't_night', 'taming', 'com_err', 'hen_vi_2', 'm_wives', 'm_for_m', 'john', 'timon', 'as_you', 'hen_viii', 'titus', 'hen_vi_1', 'win_tale', 'dream', 'pericles', 'a_and_c', 'hen_vi_3', 'rich_ii', 'merchant', 'j_caesar', 'much_ado', 'hen_iv_2', 'coriolan', 'othello', 'macbeth', 'hen_iv_1'}


### On to the actual exercises:

##### 8.1, a search engine for negation (so useful)

In [66]:
def docs_with_token(token):
    if token in index_with_freq:
        return set(index_with_freq[token]['counter'].elements())
    else:
        return set()

In [67]:
def negation_search(word):
    return doc_set.difference(docs_with_token(word))

In [72]:
print (negation_search('test'))
print (docs_with_token('test'))
print (negation_search('despair'))

{'lear', 'two_gent', 'rich_iii', 'r_and_j', 'hen_v', 'troilus', 'cymbelin', 'all_well', 'lll', 't_night', 'taming', 'com_err', 'hen_vi_2', 'm_wives', 'john', 'timon', 'as_you', 'hen_viii', 'titus', 'hen_vi_1', 'win_tale', 'dream', 'pericles', 'a_and_c', 'hen_vi_3', 'rich_ii', 'merchant', 'j_caesar', 'much_ado', 'hen_iv_2', 'coriolan', 'macbeth', 'hen_iv_1'}
{'tempest', 'hamlet', 'othello', 'm_for_m'}
{'hen_viii', 'pericles', 'two_gent', 'lll', 't_night', 'hamlet', 'm_wives', 'hen_v', 'troilus', 'j_caesar', 'much_ado', 'timon', 'othello', 'as_you', 'dream', 'hen_iv_1'}


##### 8.2 Disjunction search

Input as a list of words. Output is unordered.

In [76]:
def disjunction_search(query):
    if len(query) ==0:
        return set()
    result = docs_with_token(query[0])
    for word in query[1:]:
        result = result.union(docs_with_token(query[1]))
    return result

In [77]:
disjunction_search(['test', 'despair'])

{'a_and_c',
 'all_well',
 'com_err',
 'coriolan',
 'cymbelin',
 'hamlet',
 'hen_iv_2',
 'hen_vi_1',
 'hen_vi_2',
 'hen_vi_3',
 'john',
 'lear',
 'm_for_m',
 'macbeth',
 'merchant',
 'othello',
 'r_and_j',
 'rich_ii',
 'rich_iii',
 'taming',
 'tempest',
 'titus',
 'win_tale'}